In [2]:
!pip install wikipedia-api
!pip install matplotlib
import math
import time
import statistics
import numpy as np
import scipy.stats
import pandas as pd
import wikipediaapi
import requests
from bs4 import BeautifulSoup
import csv
import spacy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sentence_transformers import SentenceTransformer
from transformers import pipeline

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=e681de4b7508498d230d119a4508ee383c37533b849b3f7de2a6e13687513fbe
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [295]:
# from google.colab import files
uploaded = files.upload()

Saving Disney50.csv to Disney50.csv


In [296]:
# import io
sample = pd.read_csv(io.BytesIO(uploaded["Disney50.csv"]))

In [297]:
df = pd.DataFrame(sample)

Data Cleaning

In [299]:
df = df.rename(columns = {"discription" : "summary"})
df = df.drop(columns = ["Unnamed: 0"])
df = df[['character', 'traits', 'summary', 'link', 'image']]

,character,traits,summary,link,image
0,Kristoff,"direct, original, more, miserly, bashful","Kristoff was orphaned at a young age, and plac...",https://disney.fandom.com/wiki/Kristoff,https://static.wikia.nocookie.net/disney/image...
1,Simba,"direct, original, well, hard, own","As a cub, Simba was quite adventurous, inquisi...",https://disney.fandom.com/wiki/Simba,https://static.wikia.nocookie.net/disney/image...
2,Mickey Mouse,"natural, disastrous, own, tiny, derogative",Mickey's full name isMichel Mouse. He has an o...,https://disney.fandom.com/wiki/Mickey_Mouse,https://static.wikia.nocookie.net/disney/image...
3,Minnie Mouse,"prejudiced, rash, more, feminine, own","Minnie is classy, cheerful and feminine. She i...",https://disney.fandom.com/wiki/Minnie_Mouse,https://static.wikia.nocookie.net/disney/image...
4,Donald Duck,"boisterous, apathy, frightened, exaggerated, hard","According toDon Rosa's timeline, Donald Duck w...",https://disney.fandom.com/wiki/Donald_Duck,https://static.wikia.nocookie.net/disney/image...
5,Goofy,"legitimate, genuine, reliable, absent, aware","Goofy is clumsy, unintelligent, and absent-min...",https://disney.fandom.com/wiki/Goofy,https://static.wikia.nocookie.net/disney/image...
6,Pluto,"natural, loyal, kind, pal, hard","Pluto is Mickey's sidekick, best friend, consc...",https://disney.fandom.com/wiki/Pluto,https://static.wikia.nocookie.net/disney/image...
7,Daisy Duck,"later, more, well, talkative, eccentric",Daisy is shown to be a loving and caring girlf...,https://disney.fandom.com/wiki/Daisy_Duck,https://static.wikia.nocookie.net/disney/image...
8,Elsa,"light, happy, kind, original, genuine",Elsa is the queen regnant of the kingdom of Ar...,https://disney.fandom.com/wiki/Elsa,https://static.wikia.nocookie.net/disney/image...
9,Anna,"hasty, happy, more, impulse, dangerous","Unlike her older sister Elsa, Anna is optimist...",https://disney.fandom.com/wiki/Anna,https://static.wikia.nocookie.net/disney/image...


Embed all the characters

In [5]:
df = pd.read_csv('/content/drive/My Drive/disneyData50.csv')
char_embed = np.load('/content/drive/My Drive/disneyCharEmbeddings50.npy')

In [303]:
df = df.assign(rep = df["traits"] + " " + df["summary"])
model = SentenceTransformer("all-MiniLM-L6-v2")
char_embed = model.encode(list(df["rep"]))

Create function for finding most similar character

In [305]:
def find_similar(inp):
  inp_embed = model.encode([inp])
  similar_scores = model.similarity(char_embed, inp_embed)
  scores = similar_scores.tolist()
  return (df.iloc[scores.index(max(scores))], max(scores), dict(zip(df.get("character"), scores)))

Test functions

In [342]:
# for n in df.get("character"):
#   print(list(df[df.get("character") == n].get("summary")))
#   print(" ")
print(list(df[df.get("character") == "Ariel"].get("rep")))

["compassionate, happy, inspirational, original, more As a teenager, Ariel is a bright, spirited, and headstrong with a strong passion for adventure and exploration. Her tendency to explore the more mysterious depths of the ocean or more importantly, the world beyond the ocean can lead to an array of problems for both herself and her friends. Despite Ariel's headstrong nature, she is proven to be a reasonable individual, for example, she always takes responsibility for her actions."]


In [341]:
traits_inp = "passionate, artistic, empathetic"
#keywords_inp = "Gamer, Climbing, Froyo, Snow Bubble"
desc_inp = "I like music and swimming"
inp = traits_inp + " , " + desc_inp
sim_out = find_similar(inp)
#print("You are " + sim_out[0].get("character_name") + "!")
sim_out


(character                                                Ariel
 traits       compassionate, happy, inspirational, original,...
 summary      As a teenager, Ariel is a bright, spirited, an...
 link                      https://disney.fandom.com/wiki/Ariel
 image        https://static.wikia.nocookie.net/disney/image...
 rep          compassionate, happy, inspirational, original,...
 Name: 15, dtype: object,
 [0.35326313972473145],
 {'Kristoff': [0.05391595885157585],
  'Simba': [0.14004486799240112],
  'Mickey Mouse': [0.04847013205289841],
  'Minnie Mouse': [0.21280160546302795],
  'Donald Duck': [-0.03309222683310509],
  'Goofy': [0.1075284481048584],
  'Pluto': [0.09154373407363892],
  'Daisy Duck': [0.1905890703201294],
  'Elsa': [0.16294775903224945],
  'Anna': [0.1862434446811676],
  'Olaf': [0.11233440786600113],
  'Mufasa': [0.10199323296546936],
  'Nala': [0.14451400935649872],
  'Timon': [0.05682864785194397],
  'Pumbaa': [0.05770102143287659],
  'Ariel': [0.35326313972473145]

Create function for explaining similarity (ABANDONED)

In [ ]:
# generator = pipeline("text-generation", model="gpt2-medium")
# def explain(inp, char):
#   char_name, char_traits, char_keywords, char_desc = char["name"], char["traits"], char["keywords"], char["summary"]
#   prompt = f"""Compare these profiles:

# User Profile:
# Traits: {inp.split(" | ")[0]}
# Keywords: {inp.split(" | ")[1]}
# Description: {inp.split(" | ")[2]}

# Character Profile:
# Name: {char_name}
# Traits: {char_traits}
# Keywords: {char_keywords}
# Description: {char_desc}

# Task: Write a 1-sentence explanation of the similarities between the user and the character, in second person. Don't use the words I or We. If you do, you die.
# Example: Both you and Tony Stark value intelligence and determination. However, while he uses sarcasm as a defense mechanism, you tend to show more empathy in challenging situations.

# Explanation:"""

#   out = generator(
#       prompt,
#       max_new_tokens=30,
#       temperature=0.7,
#       top_p = 0.9,
#       repetition_penalty = 1.2,
#       return_full_text=False,
#   )
#   return out[0]["generated_text"].strip()
